In [1]:
# Import matplotlib.pyplot
from random import random

import matplotlib.pyplot as plt
# Import pandas
import pandas as pd
# Import geopandas
import geopandas as gpd
# import Folium
import folium
import shapely.geometry

In [7]:
# Vamos por nuestros datos
df = pd.read_csv('DATASET/recorrido_3.csv')
# Vemos su contenido
df.head()

,Coordenadas,Tipo Transporte,Coordenadas de Accidente,Tipo Accidente,Tiempo,Costo,Descripcion,Descripcion Accidente
0,"(19.383082625228962, -99.1906271466764)",VEHICULO PROPIO,Ninguna,Ninguno,05:30,$0.00,Salida,NaN
1,"(19.382573465095575, -99.19087236212239)",VEHICULO PROPIO,Ninguna,Ninguno,05:32,NaN,Giro,NaN
2,"(19.382509233732627, -99.19027091300809)",VEHICULO PROPIO,Ninguna,Ninguno,05:35,NaN,Espera para cruzar,NaN
3,"(19.382397607273372, -99.18914949367837)",VEHICULO PROPIO,Ninguna,Ninguno,05:38,NaN,Espera para cruzar,NaN
4,"(19.38227362658947, -99.1880796416457)",VEHICULO PROPIO,Ninguna,Ninguno,05:42,NaN,Espera para cruzar,NaN


In [8]:
# Vemos su descripcion
df.info()

# Vemos su tamaño mxn , m = filas, n = columnas
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Coordenadas               51 non-null     object
 1   Tipo Transporte           51 non-null     object
 2   Coordenadas de Accidente  51 non-null     object
 3   Tipo Accidente            51 non-null     object
 4   Tiempo                    51 non-null     object
 5   Costo                     1 non-null      object
 6   Descripcion               51 non-null     object
 7   Descripcion Accidente     4 non-null      object
dtypes: object(8)
memory usage: 3.3+ KB


(51, 8)

In [9]:
# Acomodamos los datos
df.Coordenadas =  df.Coordenadas.str.strip('()')
df[['Latitude', 'Longitude']] = df.Coordenadas.str.split(',', expand=True)
df.Costo =  df.Costo.str.strip('$')
# Casteamos los datos al tipo de dato
df = df.astype({'Longitude': float, 'Latitude': float, 'Costo': float})

In [10]:
# Eliminamos la columna Coordenadas
df = df.drop('Coordenadas', axis=1)
df = df.drop('Coordenadas de Accidente', axis=1)
df = df.drop('Tipo Accidente', axis=1)
df = df.drop('Descripcion Accidente', axis=1)
df.Costo = 0.0
# El resultado
df.head()

,Tipo Transporte,Tiempo,Costo,Descripcion,Latitude,Longitude
0,VEHICULO PROPIO,05:30,0.0,Salida,19.383083,-99.190627
1,VEHICULO PROPIO,05:32,0.0,Giro,19.382573,-99.190872
2,VEHICULO PROPIO,05:35,0.0,Espera para cruzar,19.382509,-99.190271
3,VEHICULO PROPIO,05:38,0.0,Espera para cruzar,19.382398,-99.189149
4,VEHICULO PROPIO,05:42,0.0,Espera para cruzar,19.382274,-99.188080


In [11]:
df.info()

df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tipo Transporte  51 non-null     object 
 1   Tiempo           51 non-null     object 
 2   Costo            51 non-null     float64
 3   Descripcion      51 non-null     object 
 4   Latitude         51 non-null     float64
 5   Longitude        51 non-null     float64
dtypes: float64(3), object(3)
memory usage: 2.5+ KB


(51, 6)

In [12]:
# Pasamos la informacion a un GeoDataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs ="EPSG:4326")

gdf['loc'] = gdf.geometry.apply(lambda x:shapely.geometry.mapping(x))
# Show
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Tipo Transporte  51 non-null     object  
 1   Tiempo           51 non-null     object  
 2   Costo            51 non-null     float64 
 3   Descripcion      51 non-null     object  
 4   Latitude         51 non-null     float64 
 5   Longitude        51 non-null     float64 
 6   geometry         51 non-null     geometry
 7   loc              51 non-null     object  
dtypes: float64(3), geometry(1), object(4)
memory usage: 3.3+ KB


In [13]:
gdf.head()

,Tipo Transporte,Tiempo,Costo,Descripcion,Latitude,Longitude,geometry,loc
0,VEHICULO PROPIO,05:30,0.0,Salida,19.383083,-99.190627,POINT (-99.19063 19.38308),"{'type': 'Point', 'coordinates': (-99.19062714..."
1,VEHICULO PROPIO,05:32,0.0,Giro,19.382573,-99.190872,POINT (-99.19087 19.38257),"{'type': 'Point', 'coordinates': (-99.19087236..."
2,VEHICULO PROPIO,05:35,0.0,Espera para cruzar,19.382509,-99.190271,POINT (-99.19027 19.38251),"{'type': 'Point', 'coordinates': (-99.19027091..."
3,VEHICULO PROPIO,05:38,0.0,Espera para cruzar,19.382398,-99.189149,POINT (-99.18915 19.38240),"{'type': 'Point', 'coordinates': (-99.18914949..."
4,VEHICULO PROPIO,05:42,0.0,Espera para cruzar,19.382274,-99.188080,POINT (-99.18808 19.38227),"{'type': 'Point', 'coordinates': (-99.18807964..."


In [14]:
cdmx = [19.432608, -99.133209]

# Creamos el mapa
m = folium.Map(location=cdmx, zoom_start=12, tiles='cartodbpositron')

In [15]:
geopath = gdf.geometry.to_json()
#print(geopath)
poligonos = folium.features.GeoJson(geopath)
m.add_child(poligonos)
folium.LayerControl().add_to(m)
m

In [16]:
m = gdf.explore(tiles="CartoDB positron")

m

In [17]:
points = gdf[['Longitude', 'Latitude']].values.tolist()


In [18]:
from numpy import sin, cos, arccos, pi, round

x = points[0]
y = points[1]

def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def getDistanceBetweenPointsNew(latitude1, longitude1, latitude2, longitude2, unit = 'miles'):

    theta = longitude1 - longitude2

    distance = 60 * 1.1515 * rad2deg(
        arccos(
            (sin(deg2rad(latitude1)) * sin(deg2rad(latitude2))) +
            (cos(deg2rad(latitude1)) * cos(deg2rad(latitude2)) * cos(deg2rad(theta)))
        )
    )

    if unit == 'miles':
        return round(distance, 2)
    if unit == 'kilometers':
        return round(distance * 1.609344, 3)


In [19]:
distances = []
for i in range(0, len(points)):
    k = i + 1
    if k == len(points):
        print("Se acabo")
    else:
        x = points[i]
        x2 = points[k]
        distances.append(getDistanceBetweenPointsNew(x[0], x[1], x2[0], x2[1], 'kilometers') * 1000)
        # print("[{0}] vs [{1}]".format(points[i], points[k]))

distances
df_distances = pd.DataFrame(distances, columns=['Distancias'])
df_distances

Se acabo


,Distancias
0,29.0
1,67.0
2,125.0
3,119.0
4,108.0
5,22.0
6,195.0
7,27.0
8,744.0
9,137.0


In [20]:
m = gdf.explore(tiles="CartoDB positron")
folium.PolyLine(gdf[['Latitude','Longitude']].values.tolist(), color="orange", weight=2.5, opacity=1).add_to(m)
m